# Profiling tutorial
1. Warning: never try to optimise your code without profiling first - it will only bring you disappointment.
2. You will probably ignore this warning the first time, just like I did.
3. Profiling takes patience, a lot of it, but it pays off and there's no way around it.

## 1. Coarse-grained profiling
Measuring the performance of an entire program run.

### /usr/bin/time
Great tool for getting an initial feel of program behaviour.
- Language independent - can measure anything
- Gives some basic data like total time spent, total memory used
- Also gives more advanced information like user/system CPU time, page faults, context switches, IO operations.
- Good for benchmarking your program: monitoring it's performance when we make changes and making sure it does not get worse.

These two examples show the behaviour of an io-intensive vs an computation-intensive program:

**MacOS** replace `-v` with `-l`

In [1]:
%%bash 
/usr/bin/time -v curl -s http://example.com > /dev/null

	Command being timed: "curl -s http://example.com"
	User time (seconds): 0.00
	System time (seconds): 0.00
	Percent of CPU this job got: 2%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.15
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 8008
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 473
	Voluntary context switches: 5
	Involuntary context switches: 2
	Swaps: 0
	File system inputs: 0
	File system outputs: 0
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


In [2]:
%%bash 
/usr/bin/time -v python -c "for i in list(range(int(1e6))): n = i"

	Command being timed: "python -c for i in list(range(int(1e6))): n = i"
	User time (seconds): 0.07
	System time (seconds): 0.00
	Percent of CPU this job got: 96%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.07
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 47752
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 8910
	Voluntary context switches: 1
	Involuntary context switches: 16
	Swaps: 0
	File system inputs: 0
	File system outputs: 0
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


The -o option will output to a file:

/usr/bin/time -v -o myfile.txt {command}

## 2. Function-level python profiling
The cProfile module will tell you how much time is spent on each function in your python program and how many times each was called.

### Timing a single block of code
For a one-off timing you can just time the function at the start and end, or use jupyter.

In [3]:
from datetime import datetime
def myfunc():
    start = datetime.now()
    n = 0
    for i in range(int(1e6)):
        n = i
    print('Time taken: {}'.format(datetime.now() - start))

myfunc()

Time taken: 0:00:00.026744


You can easily wrap this in a decorator:

In [4]:
from datetime import datetime
def time_method(func):
    def wrapper(*args, **kw):
        start_time = datetime.now()
        result = func(*args, **kw)
        end_time = datetime.now()
        print('Func: {} Took: {} secs'.format(func.__name__, (end_time - start_time).total_seconds()))
        return result
    return wrapper

@time_method
def myfunc():
    n = 0
    for i in range(int(1e6)):
        n = i

myfunc()

Func: myfunc Took: 0.026316 secs


### Using Jupyter %time
This way we can time code dead-easy:

In [5]:
def myfunc():
    n = 0
    for i in range(int(1e6)):
        n = i
% timeit myfunc()

23.5 ms ± 702 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit
n = 0
for i in range(int(1e6)):
    n = i

22.1 ms ± 25.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Memory profiling a block of code
Need to install the `memory_profiler` package: `conda install memory_profiler` or `pip install memory_profiler`

In [7]:
% load_ext memory_profiler
def myfunc():
    n = 0
    for i in range(int(1e6)):
        n = i
% memit myfunc()

peak memory: 43.49 MiB, increment: 0.04 MiB


In [8]:
%%memit
n = 0
for i in range(int(1e6)):
    n = i

peak memory: 43.50 MiB, increment: 0.00 MiB


### Using cProfile to get function call data
cProfile is included in the standard python library so no need to install anything.

In [12]:
%%bash
python -m cProfile walk.py

         409642 function calls (406338 primitive calls) in 0.965 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      210    0.001    0.000    0.001    0.000 <frozen importlib._bootstrap>:102(release)
      186    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:142(__init__)
      186    0.000    0.000    0.002    0.000 <frozen importlib._bootstrap>:146(__enter__)
      186    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap>:153(__exit__)
      210    0.001    0.000    0.001    0.000 <frozen importlib._bootstrap>:159(_get_module_lock)
      184    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:173(cb)
       24    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:178(_lock_unlock_module)
    220/3    0.000    0.000    0.169    0.056 <frozen importlib._bootstrap>:197(_call_with_frames_removed)
     1939    0.001    0.000    0.001    0.000 <frozen importlib._bootstrap

In [13]:
# Or you can call it from within python
import cProfile
from walk import keep_python_busy
cProfile.run('keep_python_busy()')

         20637769 function calls (20637766 primitive calls) in 8.100 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:989(_handle_fromlist)
        1    0.000    0.000    8.100    8.100 <string>:1(<module>)
       27    0.000    0.000    0.000    0.000 <string>:12(__new__)
        6    0.000    0.000    0.000    0.000 __init__.py:1284(debug)
        6    0.000    0.000    0.000    0.000 __init__.py:1528(getEffectiveLevel)
        6    0.000    0.000    0.000    0.000 __init__.py:1542(isEnabledFor)
        3    0.000    0.000    0.000    0.000 _collections.py:135(__init__)
       36    0.000    0.000    0.000    0.000 _collections.py:150(__getitem__)
       27    0.000    0.000    0.000    0.000 _collections.py:180(__iter__)
       24    0.000    0.000    0.000    0.000 _collections.py:208(add)
        3    0.000    0.000    0.000    0.000 _collection

        3    0.000    0.000    0.003    0.001 utils.py:630(should_bypass_proxies)
        3    0.000    0.000    0.000    0.000 utils.py:636(<lambda>)
        3    0.000    0.000    0.005    0.002 utils.py:687(get_environ_proxies)
        6    0.000    0.000    0.000    0.000 utils.py:699(select_proxy)
        3    0.000    0.000    0.000    0.000 utils.py:725(default_user_agent)
        3    0.000    0.000    0.000    0.000 utils.py:734(default_headers)
        3    0.000    0.000    0.000    0.000 utils.py:834(get_auth_from_url)
       12    0.000    0.000    0.000    0.000 utils.py:854(check_header_validity)
        3    0.000    0.000    0.000    0.000 utils.py:99(super_len)
        3    0.000    0.000    0.699    0.233 walk.py:15(download_some_stuff)
        1    4.423    4.423    5.972    5.972 walk.py:21(add_some_numbers)
        1    1.238    1.238    7.446    7.446 walk.py:30(take_a_walk_in_the_park)
        6    0.000    0.000    0.000    0.000 walk.py:5(make_random_word)
   

These results are not very readable though. Let's output them to a file so we can use some visualisation tools.

In [2]:
%%bash
python -m cProfile -o walk.prof walk.py

### Interpreting cProfile results with snakeviz

In [3]:
%%bash
snakeviz walk.prof

Process is interrupted.


### Interpreting cProfile results with kcachegrind
**Tip: ** For best visualisation results install KCacheGrind: `apt-get install kcachegrind`.

In [17]:
%%bash
pyprof2calltree -i walk.prof -k 2> /dev/null
# I added > /dev/null only so that the output does not contaminate our pretty notebook

In [18]:
# Or do it from within jupyter
from cProfile import Profile
from walk import keep_python_busy
from pyprof2calltree import convert, visualize

profiler = Profile()
profiler.run('keep_python_busy()')
visualize(profiler.getstats()) 

### Using PyCharm Pro's profiler
The "profile" button in Pycharm Pro can make it easy con do profiling-optimising iterations.

![pycharm profiling button](resources/pycharm_profile_button.png)
This will give you a call list and a call graph similar to KCacheGrind (but not interactive):
![pycharm profiling graph](resources/pycharm_profile_graph.png)

## 3. Line-level profiling
The stuff we've seen so far are great for narrowing down and finding where the bottlnecks are. However, once we have identified which functions are slow we may want to make the function faster and for that we need to know which lines are the slowest.

### Using line_profiler
We want to optimise the `take_a_walk_in_the_park` function so we want to know what lines are slowest. 

We need to specify the funtion we want to profile and the root code we want to run - they may not be the same as the function to be profiled may be called from many other functions.

In [1]:
%load_ext line_profiler

from walk import take_a_walk_in_the_park, keep_python_busy

%lprun -f take_a_walk_in_the_park keep_python_busy()

Surprisingly, computing the sum over z takes longer than calling `add_some_numbers`! When profiling, things are often non-intuitive!

### Using memory_profiler
Likewise, we may want to find out which lines in our code are incrementing our memory consumption.
You may want to restart the kernel before running this as python keeps a cache of objects.

In [5]:
%load_ext memory_profiler

from walk import take_a_walk_in_the_park, keep_python_busy

%mprun -f take_a_walk_in_the_park keep_python_busy()

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler

